In [ ]:
import bs4 as bs
import datetime as dt
import os
import pandas_datareader.data as web
import pickle
import requests
import time

In [ ]:
def get_list():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    ticks = []
    for row in table.findAll('tr')[1:]:
        tick = row.findAll('td')[0].text
        ticks.append(tick)

    tickers = ticks[0:500]
    print(len(tickers))
    with open("snp500.pickle", "wb") as f:
        pickle.dump(tickers, f)
    return tickers

tickers = get_list()

In [ ]:
def get_data():

    with open("snp500.pickle", "rb") as f:
        tickers = pickle.load(f)

    if not os.path.exists('stocks'):
        os.makedirs('stocks')

    start = dt.datetime(2016, 1, 1)
    end = dt.datetime.now()
    
    for ticker in tickers:
        temp = web.DataReader(ticker, 'iex', start, end)
        temp.to_csv('stocks/{}.csv'.format(ticker))

get_data()

In [ ]:
def convert_df():

    with open("snp500.pickle", "rb") as f:
        tickers = pickle.load(f)
       
    stocks = pd.DataFrame()
    
    for ticker in tickers:
        close = pd.read_csv('stocks/{}.csv'.format(ticker)).close
        stocks[ticker] = close
    
    lists1 = pd.read_csv('stocks/{}.csv'.format(tickers[0])).date.values
    
    stocks.index = lists1
    
    return stocks

stocks = convert_df()

In [ ]:
print(len(stocks))

stocks.head()